In [1]:
import pandas as pd
import geopandas as gpd
import requests
import json
from deep_translator import GoogleTranslator
import os

### get review data

In [30]:
key_google = os.environ.get('key_google')
key_outscraper = os.environ.get('key_outscraper')

##### get place IDs and review text via lat lon

In [3]:
url_places = 'https://places.googleapis.com/v1/places:searchNearby'

In [4]:
payload = json.dumps({
  "includedTypes": [
    "park"
  ],
  "maxResultCount": 1,
  "locationRestriction": {
    "circle": {
      "center": {
        "latitude": 52.543856,
        "longitude": 13.286259
      },
      "radius": 100
    }
  }
})

headers = {
  'X-Goog-FieldMask': 'places.id,places.displayName,places.formattedAddress,places.types,places.websiteUri',
  'Content-Type': 'application/json',
  'X-Goog-Api-Key': key_google
}

In [5]:
response = requests.request("POST", url_places, headers=headers, data=payload)

In [6]:
#ip adress need to be changed
response.json()

{'places': [{'id': 'ChIJ896tRaNWqEcReIddyp_Sa04',
   'types': ['park', 'premise', 'point_of_interest', 'establishment'],
   'formattedAddress': '13 Berlin, Germany',
   'displayName': {'text': 'Berlin', 'languageCode': 'en'}}]}

In [7]:
place_ids = [i['id'] for i in response.json()['places']]
id = place_ids[0]

In [8]:
headers = {
  'X-API-KEY': key_outscraper
}

##### get place IDs and review text text search

In [11]:
api_calls = pd.read_csv('../data/query_values.csv')

In [58]:
def textsearch (query_value, key_google):
    try:
        url_text = f'https://maps.googleapis.com/maps/api/place/textsearch/json?query={query_value}&key={key_google}'
        response = requests.request("GET", url_text)
        result = response.json().get('results')
        if result:
            place_id = result[0]['place_id']
            return place_id
        else:
            print('no results')
            return None
    except requests.RequestException as e:
        print(f"HTTP request failed: {e}")
        return None

In [59]:
api_calls['place_ids'] = api_calls['query_value'].apply(lambda x: textsearch(x, key_google))

no results
no results
no results
no results
no results
no results
no results
no results
no results


In [102]:
def check_reviews (place_id, key_google):
    
    headers = {
    'X-Goog-FieldMask': 'reviews,id,displayName,rating,location',
    f'X-Goog-Api-Key': key_google
    }
    
    try:
        url = f'https://places.googleapis.com/v1/places/{place_id}'
        response = requests.request("GET", url, headers=headers)
        result =  response.json().get('reviews')
        if result:
            return True
        else:
            return False
    except requests.RequestException as e:
        print(f"HTTP request failed: {e}")
        return None

In [100]:
test = api_calls[:10]

In [103]:
test['reviews'] = test['place_ids'].apply(lambda x: check_reviews(x, key_google))

/tmp/ipykernel_404/2498454680.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['reviews'] = test['place_ids'].apply(lambda x: check_reviews(x, key_google))


In [104]:
test

,query_value,place_ids,reviews
0,Jungfernheide/Plötzensee,ChIJZ9RRs0JRqEcR60IIK2HabF4,True
1,Schlossgarten,ChIJLQhWKi5RqEcRuLWq4wG9HDE,True
2,Jungfernheide/Plötzensee,ChIJZ9RRs0JRqEcR60IIK2HabF4,True
3,Savignyplatz,ChIJCeEX0vxQqEcRVuhtE5dxUq0,True
4,Schloßstraße,ChIJQeiNloZaqEcRkCl8zrm4Q50,False
5,Klausenerplatz,ChIJf65Iti5RqEcR_rER3tplFvk,False
6,Schloßstraße,ChIJQeiNloZaqEcRkCl8zrm4Q50,False
7,Tegeler%20Weg,ChIJwza5zDZRqEcRpnGRllTL9mw,False
8,Lietzensee,ChIJtWwWGtdQqEcRFct8sjKvQLM,True
9,Amtsgerichtsplatz,ChIJ72_VLNpQqEcRIaxo_BckNT0,False


In [81]:
headers = {
  'X-Goog-FieldMask': 'reviews,id,displayName,rating,location',
  f'X-Goog-Api-Key': key_google
}

In [82]:
url = 'https://places.googleapis.com/v1/places/ChIJ4e0dyO1QqEcRQma_KJClVKc'

In [85]:
response = requests.request("GET", url, headers=headers)
result = response.json()

In [91]:
result['reviews']

[{'name': 'places/ChIJ4e0dyO1QqEcRQma_KJClVKc/reviews/ChdDSUhNMG9nS0VJQ0FnSUNIcU5HbjZRRRAB',
  'relativePublishTimeDescription': '4 months ago',
  'rating': 5,
  'text': {'text': 'I love this place\nSpicy, fresh and delicious Asian food.\nBut unfortunately not all food kiosks accept PayPal or credit card payment methods. Most of them are “cash only”',
   'languageCode': 'en'},
  'originalText': {'text': 'I love this place\nSpicy, fresh and delicious Asian food.\nBut unfortunately not all food kiosks accept PayPal or credit card payment methods. Most of them are “cash only”',
   'languageCode': 'en'},
  'authorAttribution': {'displayName': 'Iryna Hruzda',
   'uri': 'https://www.google.com/maps/contrib/109308918531320833198/reviews',
   'photoUri': 'https://lh3.googleusercontent.com/a-/ALV-UjXEYtpmtGx_XLQOuZlbYtoLWXbMmP8E6ws-iwtmv3bngVdWdxY=s128-c0x00000000-cc-rp-mo-ba5'},
  'publishTime': '2024-08-31T15:25:21.864824Z',
  'flagContentUri': 'https://www.google.com/local/review/rap/report?

In [88]:
result.get('reviews')

[{'name': 'places/ChIJ4e0dyO1QqEcRQma_KJClVKc/reviews/ChdDSUhNMG9nS0VJQ0FnSUNIcU5HbjZRRRAB',
  'relativePublishTimeDescription': '4 months ago',
  'rating': 5,
  'text': {'text': 'I love this place\nSpicy, fresh and delicious Asian food.\nBut unfortunately not all food kiosks accept PayPal or credit card payment methods. Most of them are “cash only”',
   'languageCode': 'en'},
  'originalText': {'text': 'I love this place\nSpicy, fresh and delicious Asian food.\nBut unfortunately not all food kiosks accept PayPal or credit card payment methods. Most of them are “cash only”',
   'languageCode': 'en'},
  'authorAttribution': {'displayName': 'Iryna Hruzda',
   'uri': 'https://www.google.com/maps/contrib/109308918531320833198/reviews',
   'photoUri': 'https://lh3.googleusercontent.com/a-/ALV-UjXEYtpmtGx_XLQOuZlbYtoLWXbMmP8E6ws-iwtmv3bngVdWdxY=s128-c0x00000000-cc-rp-mo-ba5'},
  'publishTime': '2024-08-31T15:25:21.864824Z',
  'flagContentUri': 'https://www.google.com/local/review/rap/report?

##### get scrape jobs

In [71]:
id = 'ChIJZ9RRs0JRqEcR60IIK2HabF4'

In [72]:
url_scrapejobs = f'https://api.app.outscraper.com/maps/reviews-v3?limit=1000&async=false&query={id}&reviewsLimit=3'

In [73]:
response = requests.request("GET", url_scrapejobs, headers=headers)

In [74]:
response = response.json()['data']

In [75]:
response[0]

{'query': 'ChIJ4e0dyO1QqEcRQma_KJClVKc',
 'name': 'Preußenpark',
 'name_for_emails': 'Preußenpark',
 'place_id': 'ChIJ4e0dyO1QqEcRQma_KJClVKc',
 'google_id': '0x47a850edc81dede1:0xa754a59028bf6642',
 'kgmid': '/g/11bzzy741f',
 'full_address': 'Württembergische Str., 10707 Berlin, Germany',
 'borough': 'Charlottenburg-Wilmersdorf',
 'street': 'Württembergische Str.',
 'postal_code': '10707',
 'area_service': False,
 'country_code': 'DE',
 'country': 'Germany',
 'city': 'Berlin',
 'us_state': None,
 'state': None,
 'plus_code': '9F4MF8R7+JR',
 'latitude': 52.491582,
 'longitude': 13.3146104,
 'h3': '891f1d498a3ffff',
 'time_zone': 'Europe/Berlin',
 'popular_times': None,
 'site': 'https://thaipark.de/',
 'phone': '+49 30 60264358',
 'type': 'Market',
 'logo': None,
 'description': 'Tree-filled park featuring a playground & a large open space that hosts seasonal Thai-food vendors.',
 'typical_time_spent': None,
 'located_in': None,
 'located_google_id': None,
 'category': 'Markt',
 'subty

In [76]:
df = pd.DataFrame(response[0]['reviews_data'])

In [77]:
df.review_text

0    I love this place\nSpicy, fresh and delicious ...
1    Very nice concept! From Friday to Sunday you c...
2    Best place for authentic food. Very good vibes...
Name: review_text, dtype: object

In [172]:
df.columns

Index(['google_id', 'review_id', 'review_pagination_id', 'author_link',
       'author_title', 'author_id', 'author_image', 'author_reviews_count',
       'author_ratings_count', 'review_text', 'review_img_urls',
       'review_img_url', 'review_questions', 'review_photo_ids',
       'owner_answer', 'owner_answer_timestamp',
       'owner_answer_timestamp_datetime_utc', 'review_link', 'review_rating',
       'review_timestamp', 'review_datetime_utc', 'review_likes',
       'reviews_id'],
      dtype='object')

In [173]:
cols =['google_id','review_text', 'review_img_urls', 'review_rating', 'review_timestamp', 'review_likes']

In [174]:
df = df[cols]

In [176]:
df.review_rating.describe()

count    361.000000
mean       3.623269
std        1.522757
min        1.000000
25%        3.000000
50%        4.000000
75%        5.000000
max        5.000000
Name: review_rating, dtype: float64

In [196]:
df['review_text_english'] = df['review_text'].apply(lambda x: GoogleTranslator(source='de', target='en').translate(x))
#df['review_text_english'] = df['review_text_english'].apply(lambda x: GoogleTranslator(source='es', target='en').translate(x))

/tmp/ipykernel_3356/500012155.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_text_english'] = df['review_text'].apply(lambda x: GoogleTranslator(source='de', target='en').translate(x))


In [197]:
df['review_text_english']

0      I happened to visit the park around midday on ...
1      The park is nice, it surprised me that there w...
2      Legenday park in Berlin! Somehow between Kreuz...
3      I have to be honest and didn't feel particular...
4      Watched a local men’s soccer game! Looks a lit...
                             ...                        
356    Precioso de day y tentador de noche.\n\nAquí e...
357    Park that is getting old and not well maintain...
358                          Very nice park to chill 😉😏😁
359    Lots of dubious people! Especially with childr...
360    Beautiful park where you can relax when the we...
Name: review_text_english, Length: 361, dtype: object

In [202]:
df.to_csv('../data/raw/reviews_en_goerlitzerpark.csv')